###**MBA em IA E BIGDATA**
###**CURSO 2 - AM, CD E DM**
### **PROFA. ROSELI A. F. ROMERO**

##  **Dados desbalanceados**
#**OBJETIVOS**:
#- ABORDAGEM UNDERSAMPLING
#- ABORDAGEM OVERSAMPLING
#- ABORDAGEM HÍBRIDA 
---

#No dataset que será utilizado nesse exercício, cada elemento representa uma transação com cartões de crédito. 

#A base possui 30 atributos. `Time` representa o tempo em segundos desde a primeira transação da base. `Amount` corresponde ao valor da transação.

#Outros 28 atributos numéricos anonimizados também são fornecidos, com 1 variável (`Class`) de saída onde 1 significa fraude e 0 transação normal.

 

###**Questão 01**.
#Carregue o dataset `creditcard_sampled.csv` e mostre a distribuição das classes:

*   Via histograma
*   Exibindo o número de ocorrências de cada classe



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
dados = pd.read_csv('creditcard_sampled.csv', index_col=0)
dados.info()

FileNotFoundError: [Errno 2] No such file or directory: 'creditcard_sampled.csv'

#Usar a função describe() para obter: media, desvio e os quartis. Mais uma opção para o cálculo destes valores.

In [ ]:
dados.describe()

# Cálculo do Histograma

In [ ]:
dados.hist('Class')

In [ ]:
dados['Class'].value_counts()



---

### **Questão 02**.

#Separe o conjunto de dados em variáveis de entrada e variável alvo.

In [ ]:
inputs = dados.iloc[:, :-1].to_numpy()
targets = dados.iloc[:, -1].to_numpy()

print(np.bincount(targets))



---

### **Questão 03**.

#Crie uma função que aplique a técnica de subamostragem aleatória nesse conjunto de dados. 
#- Sua função deve receber como entrada a base normalizada e as classes.

#- Sua função e deve retornar um novo conjunto onde todas as classes devem ter o número de ocorrências da classe com **menor representatividade no conjunto de dados inicial** e um array com as respectivas labels.
#- Mostre o resultado plotando um histograma da distribuição resultante classes 

In [ ]:
def undersample(inputs, targets):
  # Separando os ids dos elementos que pertecem à cada classe
  classe_0_ids = np.where(targets == 0)[0]
  classe_1_ids = np.where(targets == 1)[0]

  # Guardando quantos elementos temos em cada classe
  tamanhos_classes = [len(x) for x in [classe_0_ids, classe_1_ids]]
  menor_classe = np.min(tamanhos_classes)

  # Usando numpy para fazer amostragem aleatória sem repetição (replace=false), selecionando o nro de elementos da menor classe
  # Assim teremos todos os elementos da menor classe e o mesmo nro da maior
  classe_0_sampled_ids = np.random.choice(classe_0_ids, size=menor_classe, replace=False)
  classe_1_sampled_ids = np.random.choice(classe_1_ids, size=menor_classe, replace=False)
  
  # Até agora estávamos apenas operando nos IDs agora vamos pegar os elementos selecionados no inputs
  undersampled_data = np.concatenate([inputs[classe_0_sampled_ids],
                                 inputs[classe_1_sampled_ids]])
  # O mesmo é feito para o target
  undersampled_targets = np.concatenate([targets [classe_0_sampled_ids],
                                      targets [classe_1_sampled_ids]])
  return undersampled_data, undersampled_targets

undersampled_data, undersampled_targets = undersample(inputs, targets)
plt.hist(undersampled_targets)



---

### **Questão 04**.

#Crie uma função que aplique a técnica de superamostragem aleatória nesse conjunto de dados. 

#- Sua função deve receber como entrada a base normalizada e as classes.

#- Sua função deve receber como entrada a base original e retornar um novo conjunto de dados onde todas as classes devem ter o número de ocorrências da classe com **maior representatividade no conjunto de dados inicial** e um array com as respectivas labels.
#- Mostre o resultado plotando um histograma da distribuição resultante classes 

In [ ]:
def oversample(inputs, targets):
  # Separando os ids dos elementos que pertecem à cada classe
  classe_0_ids = np.where(targets == 0)[0]
  classe_1_ids = np.where(targets == 1)[0]

  # Computando o nro de elementos de cada classe e definindo o nro de elementos
  # da classe majoritária
  tamanhos_classes = [len(x) for x in [classe_0_ids, classe_1_ids]]
  maior_classe = np.max(tamanhos_classes)

  # Se tivermos menos elementos do que a maior classe, estamos trabalhando na classe minoritária
  # Assim, queremos fazer oversample (replace=True) para aumentar o nro de elementos
  replace = False
  if tamanhos_classes[0] < maior_classe:
    replace = True
  classe_0_sampled_ids = np.random.choice(classe_0_ids, size=maior_classe, replace=replace)
  
  # Da mesma forma para a segunda classe
  replace = False
  if tamanhos_classes[1] < maior_classe:
    replace = True
  classe_1_sampled_ids = np.random.choice(classe_1_ids, size=maior_classe, replace=replace)

  # Até agora estávamos apenas operando nos IDs agora vamos pegar os elementos selecionados no inputs
  oversampled_data = np.concatenate([inputs[classe_0_sampled_ids],
                                 inputs[classe_1_sampled_ids]])
  # O mesmo é feito com o target
  oversampled_targets = np.concatenate([targets [classe_0_sampled_ids],
                                      targets [classe_1_sampled_ids]])
  return oversampled_data, oversampled_targets

oversampled_data, oversampled_targets = oversample(inputs, targets)
plt.hist(oversampled_targets)

---

### **Questão 05**.

#Crie uma função que utilize técnicas de subamostragem e superamostragem nesse conjunto de dados para balancear o número de exemplos de cada classe.

#- Sua função deve receber como entrada a base normalizada e as classes.
#- Sua função deve receber como entrada a base original e retornar uma nova base onde todas as classes devem ter o número de ocorrências **igual à $\frac{tamanho\_dataset}{3}$** e um array com as respectivas labels.
#- Mostre o resultado plotando um histograma da distribuição resultante classes 

In [ ]:
def dataset_sample(inputs, targets):
  # O número de elementos desejado é goal_size (isso também poderia vir por parâmetro)
  goal_size = int(len(targets) / 3)

  # Separando os ids dos elementos que pertecem à cada classe
  classe_0_ids = np.where(targets == 0)[0]
  classe_1_ids = np.where(targets == 1)[0]
  tamanhos_classes = [len(x) for x in [classe_0_ids, classe_1_ids]]

  # Assim como no oversample, decidimos se faremos amostragem com ou sem repetição comparando
  # o número de elementos da classe com o objetivo. Agora queremos que cada classe tenha goal_size elementos
  replace = False
  if tamanhos_classes[0] < goal_size:
    replace = True
  classe_0_sampled_ids = np.random.choice(classe_0_ids, size=goal_size, replace=replace)
  
  replace = False
  if tamanhos_classes[1] < goal_size:
    replace = True
  classe_1_sampled_ids = np.random.choice(classe_1_ids, size=goal_size, replace=replace)

  # Até agora estávamos apenas operando nos IDs agora vamos pegar os elementos selecionados no inputs
  dataset_sampled_data = np.concatenate([inputs[classe_0_sampled_ids],
                                 inputs[classe_1_sampled_ids]])
  # O mesmo para o target
  dataset_sampled_targets = np.concatenate([targets [classe_0_sampled_ids],
                                      targets [classe_1_sampled_ids]])
  return dataset_sampled_data, dataset_sampled_targets

dataset_sampled_data, dataset_sampled_targets = dataset_sample(inputs, targets)
plt.hist(dataset_sampled_targets)